# Montaña Rusa de la Era Digital 😎

In [33]:
using HTTP #Exporto las funciones
using JSON
using DataFrames
using CSV

#Pido los datos a la api
url = "https://api.n2yo.com/rest/v1/satellite/tle/25544&apiKey=T58NBR-Q5NHQE-Y76PS5-59V1"
res = HTTP.get(url)
json = JSON.parse(String(res.body))
puntos_satelites = json["tle"]


datos_1 = CSV.File("Puntos_1.csv")
datos_2 = CSV.File("Puntos_2.csv")
datos_3 = CSV.File("Puntos_3.csv")
print("Datos obtenidos de forma correcta")

4-element CSV.File:
 CSV.Row: (var"0" = 1, var"1" = 2, var"2" = 1)
 CSV.Row: (var"0" = 2, var"1" = -1, var"2" = 1)
 CSV.Row: (var"0" = 3, var"1" = 1, var"2" = -1)
 CSV.Row: (var"0" = 4, var"1" = 1, var"2" = -2)